# Quickstart

本指南介绍了如何连接到您的Milvus集群并在几分钟内执行CRUD操作

## 在开始之前

- 您已经安装[了Milvus单机版](https://milvus.io/docs/install_standalone-docker.md)或[Milvus集群版](https://milvus.io/docs/install_cluster-milvusoperator.md)。
- 您已安装首选SDK。您可以选择各种语言，包括[Python](https://milvus.io/docs/install-pymilvus.md)，[Java](https://milvus.io/docs/install-java.md)，[Go](https://milvus.io/docs/install-go.md)和[Node.js](https://milvus.io/docs/install-node.md)。

## 你好米尔乌斯

要确认您的Milvus实例可以运行并且Python SDK设置正确，请下载`hello_milvus.py`脚本。您可以使用以下命令执行此操作：

```bash
wget https://raw.githubusercontent.com/milvus-io/milvus-docs/v2.4.x/assets/hello_milvus.py
```

接下来，使用Milvus实例的地址更新脚本中的`uri`参数。更新后，使用以下命令运行脚本：

```python
python hello_milvus.py
```

如果脚本执行时没有返回任何错误消息，则说明您的Milvus实例运行正常，Python SDK安装正确。

## 连接到Milvus

一旦您获得了群集凭据或API密钥，您现在就可以使用它连接到您的Milvus。

```python
from pymilvus import MilvusClient, DataType

# 1. Set up a Milvus client
client = MilvusClient(
    uri="http://localhost:19530"
)
```

- 如果您已在Milvus实例上启用身份验证，则应在启动MilvusClient时添加`token`作为参数，并将该值设置为冒号分隔的用户名和密码。要使用默认用户名和密码进行身份验证，请将`token`设置为`root:Milvus`。有关如何启用身份验证的信息，请参阅[验证用户访问](https://milvus.io/docs/authenticate.md)。
- 如果您已在Milvus实例上启用TLS，则应在启动MilvusClient时提供证书文件路径。有关如何启用TLS的信息，请参阅[传输中的加密](https://milvus.io/docs/tls.md)。

## 创建集合

在Milvus中，您需要将向量嵌入存储在集合中。存储在集合中的所有向量嵌入共享相同的维度和距离度量来测量相似性。您可以通过以下任一方式创建集合。

### 快速设置

要在快速设置模式下设置集合，您只需要设置集合名称和集合的向量字段的维度。

```python
# 2. Create a collection in quick setup mode
client.create_collection(
    collection_name="quick_setup",
    dimension=5
)
```

在上述设置中，

- 主字段和向量字段使用它们的默认名称（id和vector）。
- 度量类型也设置为默认值（COSINE）。
- 主字段接受整数，不会自动递增。
- 名为$Meta的JSON保留字段用于存储非模式定义的字段及其值。

**注意到**

使用RESTful API创建的集合支持至少32维向量字段。

### 自定义设置

要自己定义集合架构，请使用自定义设置。通过这种方式，可以定义集合中每个字段的属性，包括其名称、数据类型和特定字段的额外属性。

```python
# 3. Create a collection in customized setup mode

# 3.1. Create schema
schema = MilvusClient.create_schema(
    auto_id=False,
    enable_dynamic_field=True,
)

# 3.2. Add fields to schema
schema.add_field(field_name="my_id", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="my_vector", datatype=DataType.FLOAT_VECTOR, dim=5)

# 3.3. Prepare index parameters
index_params = client.prepare_index_params()

# 3.4. Add indexes
index_params.add_index(
    field_name="my_id"
)

index_params.add_index(
    field_name="my_vector", 
    index_type="AUTOINDEX",
    metric_type="IP"
)

# 3.5. Create a collection
client.create_collection(
    collection_name="customized_setup",
    schema=schema,
    index_params=index_params
)
```

在上面的设置中，您可以在创建集合的过程中灵活地定义集合的各个方面，包括它的模式和索引参数。

- **模式**

  架构定义了集合的结构。除了如上所述添加预定义字段和设置其属性外，您还可以选择启用和禁用

  - **AutoID**

    ```undefined
    Whether to enable the collection to automatically increment the primary field.
    ```

  - **动力场**

    ```javascript
    Whether to use the reserved JSON field __$meta__ to store non-schema-defined fields and their values.
    ```

    有关架构的详细说明，请参阅架构。

- **指标参数**

  索引参数决定了Milvus如何在集合中组织数据。您可以通过配置字段的度量类型和索引类型来为字段分配特定索引。

  - 对于向量字段，您可以使用AUTOINDEX作为索引类型，并使用COSINE、L2或IP作为`metric_type`。

  - 对于标量字段，包括主字段，Milvus对整数使用TRIE，对字符串使用STL_SORT。

    有关索引类型的更多信息，请参阅[索引](https://milvus.io/docs/index.md)。

**注意到**

自动加载在前面的代码段中创建的集合。如果您不希望创建自动加载的收藏，请参阅[管理收藏](https://milvus.io/docs/manage-collections.md)。

使用RESTful API创建的集合总是自动加载的。

## 插入数据

以上述任何一种方式创建的集合都已被索引和加载。一旦你准备好了，插入一些示例数据。

```python
# 4. Insert data into the collection
# 4.1. Prepare data
data=[
    {"id": 0, "vector": [0.3580376395471989, -0.6023495712049978, 0.18414012509913835, -0.26286205330961354, 0.9029438446296592], "color": "pink_8682"},
    {"id": 1, "vector": [0.19886812562848388, 0.06023560599112088, 0.6976963061752597, 0.2614474506242501, 0.838729485096104], "color": "red_7025"},
    {"id": 2, "vector": [0.43742130801983836, -0.5597502546264526, 0.6457887650909682, 0.7894058910881185, 0.20785793220625592], "color": "orange_6781"},
    {"id": 3, "vector": [0.3172005263489739, 0.9719044792798428, -0.36981146090600725, -0.4860894583077995, 0.95791889146345], "color": "pink_9298"},
    {"id": 4, "vector": [0.4452349528804562, -0.8757026943054742, 0.8220779437047674, 0.46406290649483184, 0.30337481143159106], "color": "red_4794"},
    {"id": 5, "vector": [0.985825131989184, -0.8144651566660419, 0.6299267002202009, 0.1206906911183383, -0.1446277761879955], "color": "yellow_4222"},
    {"id": 6, "vector": [0.8371977790571115, -0.015764369584852833, -0.31062937026679327, -0.562666951622192, -0.8984947637863987], "color": "red_9392"},
    {"id": 7, "vector": [-0.33445148015177995, -0.2567135004164067, 0.8987539745369246, 0.9402995886420709, 0.5378064918413052], "color": "grey_8510"},
    {"id": 8, "vector": [0.39524717779832685, 0.4000257286739164, -0.5890507376891594, -0.8650502298996872, -0.6140360785406336], "color": "white_9381"},
    {"id": 9, "vector": [0.5718280481994695, 0.24070317428066512, -0.3737913482606834, -0.06726932177492717, -0.6980531615588608], "color": "purple_4976"}
]

# 4.2. Insert data
res = client.insert(
    collection_name="quick_setup",
    data=data
)

print(res)

# Output
#
# {
#     "insert_count": 10,
#     "ids": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
# }
```

提供的代码假定您已经以快速设置方式创建了集合。如上面的代码所示，

- 要插入的数据被组织到一个字典列表中，其中每个字典表示一个数据记录，称为实体。
- 每个字典都包含一个名为color的非模式定义字段。
- 每个字典都包含与预定义字段和动态字段对应的键。

**注意到**

使用RESTful API启用AutoID创建的集合，因此您需要跳过数据中的主字段以插入。

### 插入更多数据

如果您希望稍后使用插入的10个图元进行搜索，则可以安全地跳过此部分。要了解有关Milvus搜索性能的更多信息，建议您使用以下代码片段将更多随机生成的实体添加到集合中。

```python
# 5. Insert more data into the collection
# 5.1. Prepare data

colors = ["green", "blue", "yellow", "red", "black", "white", "purple", "pink", "orange", "brown", "grey"]
data = [ {
    "id": i, 
    "vector": [ random.uniform(-1, 1) for _ in range(5) ], 
    "color": f"{random.choice(colors)}_{str(random.randint(1000, 9999))}" 
} for i in range(1000) ]

# 5.2. Insert data
res = client.insert(
    collection_name="quick_setup",
    data=data[10:]
)

print(res)

# Output
#
# {
#     "insert_count": 990
# }
```

**注意到**

每次调用InsertRESTful API时，最多可以在一个批中插入100个实体。

## 相似性搜索

您可以基于一个或多个向量嵌入进行相似性搜索。

**注意到**

插入操作是异步的，在数据插入之后立即执行搜索可能会导致空结果集。为了避免这种情况，建议您等待几秒钟。

### 单向量搜索

query_vectors变量的值是一个包含浮点数子列表的列表。子列表表示5维的向量嵌入。

```python
# 6. Search with a single vector
# 6.1. Prepare query vectors
query_vectors = [
    [0.041732933, 0.013779674, -0.027564144, -0.013061441, 0.009748648]
]

# 6.2. Start search
res = client.search(
    collection_name="quick_setup",     # target collection
    data=query_vectors,                # query vectors
    limit=3,                           # number of returned entities
)

print(res)

# Output
#
# [
#     [
#         {
#             "id": 548,
#             "distance": 0.08589144051074982,
#             "entity": {}
#         },
#         {
#             "id": 736,
#             "distance": 0.07866684347391129,
#             "entity": {}
#         },
#         {
#             "id": 928,
#             "distance": 0.07650312781333923,
#             "entity": {}
#         }
#     ]
# ]
```

输出是一个包含三个字典的子列表的列表，表示返回的实体及其ID和距离。

### 体向量搜索

您还可以在query_vectors变量中包含多个向量嵌入，以进行批量相似性搜索。

```python
# 7. Search with multiple vectors
# 7.1. Prepare query vectors
query_vectors = [
    [0.041732933, 0.013779674, -0.027564144, -0.013061441, 0.009748648],
    [0.0039737443, 0.003020432, -0.0006188639, 0.03913546, -0.00089768134]
]

# 7.2. Start search
res = client.search(
    collection_name="quick_setup",
    data=query_vectors,
    limit=3,
)

print(res)

# Output
#
# [
#     [
#         {
#             "id": 548,
#             "distance": 0.08589144051074982,
#             "entity": {}
#         },
#         {
#             "id": 736,
#             "distance": 0.07866684347391129,
#             "entity": {}
#         },
#         {
#             "id": 928,
#             "distance": 0.07650312781333923,
#             "entity": {}
#         }
#     ],
#     [
#         {
#             "id": 532,
#             "distance": 0.044551681727170944,
#             "entity": {}
#         },
#         {
#             "id": 149,
#             "distance": 0.044386886060237885,
#             "entity": {}
#         },
#         {
#             "id": 271,
#             "distance": 0.0442606583237648,
#             "entity": {}
#         }
#     ]
# ]
```

输出应该是两个子列表的列表，每个子列表包含三个字典，表示返回的实体及其ID和距离。

### 过滤搜索

- **使用模式定义的字段**

  您还可以通过在搜索请求中包含筛选器和指定某些输出字段来增强搜索结果。

  ```python
  # 8. Search with a filter expression using schema-defined fields
  # 1 Prepare query vectors
  query_vectors = [
      [0.041732933, 0.013779674, -0.027564144, -0.013061441, 0.009748648]
  ]
  
  # 2. Start search
  res = client.search(
      collection_name="quick_setup",
      data=query_vectors,
      filter="500 < id < 800",
      limit=3
  )
  
  print(res)
  
  # Output
  #
  # [
  #     [
  #         {
  #             "id": 548,
  #             "distance": 0.08589144051074982,
  #             "entity": {}
  #         },
  #         {
  #             "id": 736,
  #             "distance": 0.07866684347391129,
  #             "entity": {}
  #         },
  #         {
  #             "id": 505,
  #             "distance": 0.0749310627579689,
  #             "entity": {}
  #         }
  #     ]
  # ]
  ```

  输出应该是一个包含三个字典的子列表的列表，每个字典代表一个搜索到的实体及其ID、距离和指定的输出字段。

- **使用非模式定义的字段**

  还可以在筛选表达式中包含动态字段。在下面的代码片段中，`color`是非模式定义的字段。您可以将它们作为键包含在神奇的`$meta`字段中，例如`$meta["color"]`，或者直接将其用作模式定义的字段，例如`color`。

  ```python
  # 9. Search with a filter expression using custom fields
  # 9.1.Prepare query vectors
  query_vectors = [
      [0.041732933, 0.013779674, -0.027564144, -0.013061441, 0.009748648]
  ]
  
  # 9.2.Start search
  res = client.search(
      collection_name="quick_setup",
      data=query_vectors,
      filter='$meta["color"] like "red%"',
      limit=3,
      output_fields=["color"]
  )
  
  print(res)
  
  # Output
  #
  # [
  #     [
  #         {
  #             "id": 240,
  #             "distance": 0.0694073885679245,
  #             "entity": {
  #                 "color": "red_8667"
  #             }
  #         },
  #         {
  #             "id": 581,
  #             "distance": 0.059804242104291916,
  #             "entity": {
  #                 "color": "red_1786"
  #             }
  #         },
  #         {
  #             "id": 372,
  #             "distance": 0.049707964062690735,
  #             "entity": {
  #                 "color": "red_2186"
  #             }
  #         }
  #     ]
  # ]
  ```

## 标量查询

与向量相似性搜索不同，查询通过基于[过滤器表达式的](https://milvus.io/docs/boolean.md)标量过滤来检索向量。

- **使用架构定义的字段进行筛选**

  ```python
  # 10. Query with a filter expression using a schema-defined field
  res = client.query(
      collection_name="quick_setup",
      filter="10 < id < 15",
      output_fields=["color"]
  )
  
  print(res)
  
  # Output
  #
  # [
  #     {
  #         "color": "green_7413",
  #         "id": 11
  #     },
  #     {
  #         "color": "orange_1417",
  #         "id": 12
  #     },
  #     {
  #         "color": "orange_6143",
  #         "id": 13
  #     },
  #     {
  #         "color": "white_4084",
  #         "id": 14
  #     }
  # ]
  ```

## 获取实体

如果知道要检索的实体的ID，则可以按如下方式通过实体的ID获取实体：

```python
# 12. Get entities by IDs
res = client.get(
    collection_name="quick_setup",
    ids=[1,2,3],
    output_fields=["color", "vector"]
)

print(res)

# Output
#
# [
#     {
#         "id": 1,
#         "color": "red_7025",
#         "vector": [
#             0.19886813,
#             0.060235605,
#             0.6976963,
#             0.26144746,
#             0.8387295
#         ]
#     },
#     {
#         "id": 2,
#         "color": "orange_6781",
#         "vector": [
#             0.43742132,
#             -0.55975026,
#             0.6457888,
#             0.7894059,
#             0.20785794
#         ]
#     },
#     {
#         "id": 3,
#         "color": "pink_9298",
#         "vector": [
#             0.3172005,
#             0.97190446,
#             -0.36981148,
#             -0.48608947,
#             0.9579189
#         ]
#     }
# ]
```

**注意到**

目前，RESTful API不提供get端点。

## 删除实体

Milvus允许通过ID和过滤器删除实体。

- **按ID删除实体。**

  ```python
  # 13. Delete entities by IDs
  res = client.delete(
      collection_name="quick_setup",
      ids=[0,1,2,3,4]
  )
  
  print(res)
  
  # Output
  #
  # {
  #     "delete_count": 5
  # }
  ```

- **按筛选器删除实体**

  ```python
  # 14. Delete entities by a filter expression
  res = client.delete(
      collection_name="quick_setup",
      filter="id in [5,6,7,8,9]"
  )
  
  print(res)
  
  # Output
  #
  # {
  #     "delete_count": 5
  # }
  ```

  **注意到**

  目前，RESTful API的删除端点不支持过滤器。

## 放下收藏

Starter计划最多允许在无服务器集群中进行两次收集。完成本指南后，您可以按如下方式删除集合：

```python
# 15. Drop collection
client.drop_collection(
    collection_name="quick_setup"
)

client.drop_collection(
    collection_name="customized_setup"
)
```

## Recaps

- 有两种方法可以创建一个集合。第一种是快速设置，它只需要您提供向量场的名称和维度。第二个是自定义设置，它允许您自定义集合的几乎每个方面。
- 数据插入过程可能需要一些时间才能完成。建议在插入数据后等待几秒钟，然后再进行相似性搜索。
- 筛选表达式可用于搜索和查询请求。但是，它们对于查询请求是强制性的。